In [41]:
import numpy as np
import pandas as pd


In [42]:
data = pd.read_csv('50_Startups.csv')

In [43]:
features = data.iloc[:,:-1].values
label = data.iloc[:,[-1]].values

In [44]:
data.describe()

,R&D Spend,Administration,Marketing Spend,Profit
count,50.000000,50.000000,50.000000,50.000000
mean,73721.615600,121344.639600,211025.097800,112012.639200
std,45902.256482,28017.802755,122290.310726,40306.180338
min,0.000000,51283.140000,0.000000,14681.400000
25%,39936.370000,103730.875000,129300.132500,90138.902500
50%,73051.080000,122699.795000,212716.240000,107978.190000
75%,101602.800000,144842.180000,299469.085000,139765.977500
max,165349.200000,182645.560000,471784.100000,192261.830000


In [45]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
encoder = LabelEncoder()
features[:,3] = encoder.fit_transform(features[:,3])
ohe = OneHotEncoder(categorical_features=[3])
features = ohe.fit_transform(features).toarray()

In [46]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(features,
                                                label,
                                                test_size=0.2,
                                                random_state=1)

In [47]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [48]:
model.score(X_train,y_train)

0.9424465426893971

In [49]:
model.score(X_test,y_test)

0.9649618042060633

In [51]:
features[0]

array([0.000000e+00, 0.000000e+00, 1.000000e+00, 1.653492e+05,
       1.368978e+05, 4.717841e+05])

In [52]:
encoder.classes_

array(['California', 'Florida', 'New York'], dtype=object)

In [63]:
rdspend = float(input('Enter R&D Spend: '))
adspend = float(input('Enter Admin Spend: '))
marketing = float(input('Enter Marketing Spend: '))
state = input('Enter State: ')

if state in encoder.classes_:
    labelState = encoder.transform(np.array([state]))
    features = np.array([[rdspend,adspend,marketing,labelState]])
    finalFeatures = ohe.transform(features).toarray()
    print("Profit is ",model.predict(finalFeatures))
else:
    print("Model is not aware of {} state".format(state))


Enter R&D Spend: 4566666
Enter Admin Spend: 54555
Enter Marketing Spend: 5666666
Enter State: California
Profit is  [[3749066.58333232]]


# OLS Backward Elimination Technique


In [32]:
features[0]

array([0.000000e+00, 0.000000e+00, 1.000000e+00, 1.653492e+05,
       1.368978e+05, 4.717841e+05])

In [35]:
#Preparing the features
features  = np.append( np.ones((50,1)).astype(int) , features , axis = 1 )

In [36]:
features[0]

array([1.000000e+00, 0.000000e+00, 0.000000e+00, 1.000000e+00,
       1.653492e+05, 1.368978e+05, 4.717841e+05])

In [12]:
encoder.classes_

array(['California', 'Florida', 'New York'], dtype=object)

In [11]:
features[0]

array([1.000000e+00, 0.000000e+00, 0.000000e+00, 1.000000e+00,
       1.653492e+05, 1.368978e+05, 4.717841e+05])

1. b0 = 1
2. b1 = 'California'
3. b2 = 'Florida'
4. b3 =  'New York'
5. b4 = 'R&D Spend'
6. b5 = 'Admin'
7. b6 = 'Marketing'

In [10]:
import statsmodels.formula.api as sm
#Creating an OLS model ---- LinearRegressionModel
#Step 1 -- All in (Pass all the feature variable available along with the label)
# endog ----> label , exog ----> features
#Iteration 1:
model1 = sm.OLS(endog=label , exog=features).fit() #to create equation
model1.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.951
Model:                            OLS   Adj. R-squared:                  0.945
Method:                 Least Squares   F-statistic:                     169.9
Date:                Mon, 03 Jun 2019   Prob (F-statistic):           1.34e-27
Time:                        19:04:32   Log-Likelihood:                -525.38
No. Observations:                  50   AIC:                             1063.
Df Residuals:                      44   BIC:                             1074.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       3.763e+04   5073.636      7.417      0.000    2.74e+04    4.79e+04
x1          1.249e+04   2449.797      5.099      0.000    7554.868    1.74e+04
x2          1.269e+04   2726.700      4.654      0.000    7195.596    1.82e+04
x3          1.245e+04   2486.364      5.007      0.000    7439.285    1.75e+04
x4             0.8060      0.046     17.369      0.000       0.712       0.900
x5            -0.0270      0.052     -0.517      0.608      -0.132       0.078
x6             0.0270      0.017      1.574      0.123      -0.008       0.062
==============================================================================
Omnibus:                       14.782   Durbin-Watson:                   1.283
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               21.266
Skew:                          -0.948   Prob(JB):                     2.41e-05
Kurtosis:                       5.572   Cond. No.                     1.81e+17
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 1.19e-22. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

# Observation is to eliminate x6 - Marketing

In [14]:
#Iteration 2
newFeatures1 = features[:,[0,1,2,3,4,5]]
model1 = sm.OLS(endog=label , exog=newFeatures1).fit() #to create equation
model1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.948
Model:                            OLS   Adj. R-squared:                  0.943
Method:                 Least Squares   F-statistic:                     205.0
Date:                Mon, 03 Jun 2019   Prob (F-statistic):           2.90e-28
Time:                        19:10:06   Log-Likelihood:                -526.75
No. Observations:                  50   AIC:                             1064.
Df Residuals:                      45   BIC:                             1073.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       4.122e+04   4607.941      8.945      0.000    3.19e+04    5.05e+04
x1          1.339e+04   2421.500      5.529      0.000    8511.111    1.83e+04
x2          1.448e+04   2518.987      5.748      0.000    9405.870    1.96e+04
x3          1.335e+04   2459.306      5.428      0.000    8395.623    1.83e+04
x4             0.8609      0.031     27.665      0.000       0.798       0.924
x5            -0.0527      0.050     -1.045      0.301      -0.154       0.049
==============================================================================
Omnibus:                       14.275   Durbin-Watson:                   1.197
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               19.260
Skew:                          -0.953   Prob(JB):                     6.57e-05
Kurtosis:                       5.369   Cond. No.                     3.34e+17
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 9.69e-24. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

# Observation is to eliminate Administration

In [17]:
#Iteration 3
newFeatures2 = newFeatures1[:,[0,1,2,3,4]]
model1 = sm.OLS(endog=label , exog=newFeatures2).fit() #to create equation
model1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.947
Model:                            OLS   Adj. R-squared:                  0.943
Method:                 Least Squares   F-statistic:                     272.4
Date:                Mon, 03 Jun 2019   Prob (F-statistic):           2.76e-29
Time:                        19:11:22   Log-Likelihood:                -527.35
No. Observations:                  50   AIC:                             1063.
Df Residuals:                      46   BIC:                             1070.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       3.686e+04   1959.786     18.806      0.000    3.29e+04    4.08e+04
x1          1.189e+04   1956.677      6.079      0.000    7955.697    1.58e+04
x2          1.306e+04   2122.665      6.152      0.000    8785.448    1.73e+04
x3           1.19e+04   2036.022      5.847      0.000    7805.580     1.6e+04
x4             0.8530      0.030     28.226      0.000       0.792       0.914
==============================================================================
Omnibus:                       13.418   Durbin-Watson:                   1.122
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               17.605
Skew:                          -0.907   Prob(JB):                     0.000150
Kurtosis:                       5.271   Cond. No.                     3.20e+17
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 3.66e-24. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

# All Elimination Done

In [18]:
from sklearn.model_selection import train_test_split
X_train_final,X_test_final,y_train_final,y_test_final = train_test_split(newFeatures2,
                                                                        label,
                                                                        test_size=0.2,
                                                                        random_state=1)

In [19]:
from sklearn.linear_model import LinearRegression
finalModel = LinearRegression()
finalModel.fit(X_train_final,y_train_final)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [20]:
finalModel.score(X_train_final,y_train_final)

0.9387161169637389

In [21]:
finalModel.score(X_test_final,y_test_final)

0.9611102529882911

## The above is the final model that we can deploy based on OLS Backward Elimination Technique